In [7]:
import fiona
import rasterio
import numpy as np
from rasterio.features import rasterize
from fiona.transform import transform as f_transform
import pyproj
import shapely
from shapely import ops

In [2]:
nomenclature = {
    'culture-prairie': 1,
    'eau': 2,
    'foret': 3,
    'foret_plantee': 4,
    'friche': 5,
    'urbain et artificiel': 6,
    'verger': 7,
    'voie de communication': 8
}

In [4]:
shapefile = fiona.open("/mnt/d/pfournie/Documents/data/alumhi/OCS_1946_210714_VERSION_FINALE_Merge.shp", 'r')
raster = rasterio.open('/mnt/d/pfournie/Documents/data/alumhi/1946-3.tif', 'r')

shapes1946 = [(s['geometry'], nomenclature[s['properties']['OCS_1946']]) for s in shapefile]

meta = raster.meta.copy()
meta.update(compress='lzw')
meta.update(count=1)
with rasterio.open('/mnt/d/pfournie/Documents/data/alumhi/1946-labels.tif', 'w+', **meta) as out:
    out_arr = out.read(1) 
    burned = rasterize(shapes=shapes1946, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

In [5]:
shapefile = fiona.open("/mnt/d/pfournie/Documents/data/alumhi/OCS_2019_210712_VERSION_FINALE_Merge.shp", 'r')
raster = rasterio.open('/mnt/d/pfournie/Documents/data/alumhi/data_2019_20cm.tif', 'r')

In [8]:
epsg2154 = pyproj.CRS('EPSG:2154')
epsg4326 = pyproj.CRS('EPSG:4326')
project = pyproj.Transformer.from_crs(epsg2154, epsg4326, always_xy=True).transform
poly2154 = shapely.Polygon(next(iter(shapefile))['geometry']['coordinates'][0][0])
print(poly2154)
poly4326 = ops.transform(project, poly2154)
print(poly4326)

POLYGON ((703048.4157122001 6500796.28226898, 703074.2748366594 6500793.18308772, 703089.2222749963 6500797.286964571, 703107.5720454454 6500791.087277344, 703087.953278102 6500781.989119809, 703082.5326902121 6500759.778640761, 702995.3676260039 6500791.550942904, 703021.1263213903 6500802.943679454, 703048.4157122001 6500796.28226898))
POLYGON ((3.0391092789427745 45.60668163230292, 3.039441015522701 45.60665360904057, 3.0396328081636512 45.60669049615037, 3.039868184152981 45.60663458615616, 3.0396164292553545 45.60655274817484, 3.0395467439456243 45.60635277219256, 3.0384286755347607 45.60663926229162, 3.0387592150254457 45.606741737997446, 3.0391092789427745 45.60668163230292))


In [9]:
shapes2019 = []
for s in shapefile:
    label = nomenclature[s['properties']['OCS_2019']]
    print(label)
    for coords in s['geometry']['coordinates']:
        poly2154 = shapely.Polygon(coords[0])
        poly4326 = shapely.ops.transform(project, poly2154)
        shapes2019.append((poly4326, label))
print(len(shapes2019))

5
3
7
4
6
1
2
8
3728


In [10]:
meta = raster.meta.copy()
meta.update(compress='lzw')
meta.update(count=1)
with rasterio.open('/mnt/d/pfournie/Documents/data/alumhi/2019-labels.tif', 'w+', **meta) as out:
    out_arr = out.read(1) 
    burned = rasterize(shapes=shapes2019, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

In [ ]:
#minx, miny, maxx, maxy = shapefile.bounds
#(minx, maxx), (miny, maxy) = f_transform(shapefile.crs, raster.crs, [minx, maxx], [miny, maxy])
#w = rasterio.windows.from_bounds(minx, miny, maxx, maxy, transform=raster.transform)

#minx, miny, maxx, maxy = rasterio.windows.bounds(w, transform=raster.transform)
#t = rasterio.transform.from_bounds(minx, miny, maxx, maxy, 2000, 2000)
#(minx, maxx), (miny, maxy) = f_transform(raster.crs, shapefile.crs, [minx, maxx], [miny, maxy])
#
#profile.update(dtype=rasterio.uint8, count=3, width=2000, height=2000, transform=t, compress='lzw')
#with rasterio.Env():
#    with rasterio.open('/mnt/d/pfournie/Documents/data/alumhi/1946-3-labels.tif', 'w', **profile) as dst:
#        for i, att in enumerate(['num_class']):
#            shapes_for_raster = [(s['geometry'], s['properties'][att]) for s in tile_shp.filter(bbox=(minx, miny, maxx, maxy))]
#            masks = rasterize(shapes_for_raster, out_shape=(2000, 2000), transform=t)
#            dst.write(masks.astype(rasterio.uint8), i+1)